I decided to make an AI tutor 

In [2]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [3]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [4]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [5]:
system_prompt = """You are the software engineer, phd in mathematics, machine learning engineer, and other topics"""
system_prompt += """
When responding, always use Markdown for formatting. For any code, use well-structured code blocks with syntax highlighting,
For instance:
```python

sample_list = [for i in range(10)]
```
Another example
```javascript
        function displayMessage() {
            alert("Hello, welcome to JavaScript!");
        }

```

Break down explanations into clear, numbered steps for better understanding. 
Highlight important terms using inline code formatting (e.g., `function_name`, `variable`).
Provide examples for any concepts and ensure all examples are concise, clear, and relevant.
Your goal is to create visually appealing, easy-to-read, and informative responses.

"""


In [6]:
def tutor_user_prompt(question):
    # Ensure the question is properly appended to the user prompt.
    user_prompt = (
        "Please carefully explain the following question in a step-by-step manner for clarity:\n\n"
    )
    user_prompt += question
    return user_prompt

In [10]:

def askTutor(question, MODEL):
    # Generate the user prompt dynamically.
    user_prompt = tutor_user_prompt(question)
    
    # OpenAI API call to generate response.
    if MODEL == 'gpt-4o-mini':
        print(f'You are getting response from {MODEL}')
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )
    else:
        MODEL == 'llama3.2'
        print(f'You are getting response from {MODEL}')
        stream = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )

    # Initialize variables for response processing.
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Process the response stream and update display dynamically.
    for chunk in stream:
      
         response += chunk.choices[0].delta.content or ''
         update_display(Markdown(response), display_id=display_handle.display_id)


In [11]:

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [12]:
askTutor(question=question, MODEL=MODEL_GPT)

You are getting response from gpt-4o-mini


Certainly! Let's break down the provided code snippet step by step for clarity:

### Code Overview

The code is a Python expression that utilizes the `yield from` statement along with a generator expression. Here’s the code for reference:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Breakdown Steps

1. **Understanding `yield from`:**
   - The `yield from` statement is used in a generator function to yield all values from an iterable (like a list or a set) without needing to loop through it manually.
   - It essentially delegates the yielding of values to another iterator.

2. **Comprehension `{...}`:**
   - This code uses a **set comprehension** to create a set (denoted by `{}`) of authors.
   - A set is a collection of unique items, so any duplicate authors will appear only once in the resulting set.

3. **The `for` Loop:**
   - Inside the comprehension, we have a loop that iterates over `books`.
   - `books` is presumably a list or collection of dictionaries where each dictionary represents a book (with attributes like "author").

4. **The `if` Condition:**
   - The condition `if book.get("author")` checks if the `author` key exists in the `book` dictionary and if it holds a value that is not `None` or an empty string.
   - This means that only books with a valid author will be included in the set.

5. **The `get()` Method:**
   - `book.get("author")` is a method call that retrieves the value associated with the `author` key from each `book` dictionary.
   - If the key is not found, `get()` will return `None`, ensuring that books without an author are excluded by the previous condition.

### Summary of Functionality

Putting it all together, the complete line of code does the following:

1. Iterates over a collection of `books`.
2. Extracts the `author` value for each book that has a valid `author`.
3. Accumulates these authors into a **set**, ensuring all authors are unique.
4. Yields each author from the set one by one.

### Example

Let's visualize this with an example:

```python
# Example list of books
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": None},
    {"title": "Book Four", "author": "Author A"},
]

# Generator function using your code
def unique_authors():
    yield from {book.get("author") for book in books if book.get("author")}

# Using the generator
for author in unique_authors():
    print(author)
```

### Expected Output

For the above example, the output would be:

```
Author A
Author B
```

### Conclusion

- The code effectively retrieves all unique authors from a list of books, skipping any that do not have an author value.
- This approach is efficient and leverages Python’s set comprehensions and generator functionality effectively.

2. Using both LLMs collaboratively approach

I thought about like similar the idea of a RAG (Retrieval-Augmented Generation) approach, is an excellent idea to improve responses by refining the user query and producing a polished, detailed final answer. Two LLM talking each other its cool!!! Here's how we can implement this:
Updated Concept:

Refine Query with Ollama:
Use Ollama to refine the raw user query into a well-structured prompt.
This is especially helpful when users input vague or poorly structured queries.
Generate Final Response with GPT:
Pass the refined prompt from Ollama to GPT to generate the final, detailed, and polished response.
Return the Combined Output:
Combine the input, refined query, and the final response into a single display to ensure clarity.

In [13]:
def refine_with_ollama(raw_question):
    """
    Use Ollama to refine the user's raw question into a well-structured prompt.
    """
    print("Refining the query using Ollama...")
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Refine and structure the following user input."},

        {"role": "user", "content": raw_question},
    ]
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
        stream=False  # Non-streamed refinement
    )
    refined_query = response.choices[0].message.content
    return refined_query

In [17]:
def ask_with_ollama_and_gpt(raw_question):
    """
    Use Ollama to refine the user query and GPT to generate the final response.
    """
    # Step 1: Refine the query using Ollama
    refined_query = refine_with_ollama(raw_question)
    
    # Step 2: Generate final response with GPT
    print("Generating the final response using GPT...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": refined_query},
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True  # Stream response for dynamic display
    )

    # Step 3: Combine responses
    response = ""
    display_handle = display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:"), display_id=True)
    for chunk in stream:
        response_chunk = chunk.choices[0].delta.content or ""

        update_display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:\n\n{response}"), display_id=display_handle.display_id)

In [18]:
# Example Usage
question = """
Please explain what this code does:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
ask_with_ollama_and_gpt(raw_question=question)

Refining the query using Ollama...
